In [1]:
#!pip install azure-cognitiveservices-vision-customvision

In [2]:
from azure.cognitiveservices.vision.customvision.training import CustomVisionTrainingClient
from azure.cognitiveservices.vision.customvision.training.models import *
from msrest.authentication import ApiKeyCredentials 

In [4]:
# Setup credentials and ENDPOINT for the related service.
ENDPOINT = # set your own
training_key = # set your own
credentials = ApiKeyCredentials(in_headers={"Training-key": training_key})
project_id = # set your own

# Create the training client.
trainer = CustomVisionTrainingClient(ENDPOINT, credentials)

# Function checking if the model needs to be retrained and do so.
def retrain():
    # Set project id (fetched from the customvision console (website))

    # Get the last iteration
    iteration_id = trainer.get_iterations(project_id)[0].id

    tagIds = trainer.get_tags(project_id, iteration_id)
    for tag in tagIds:
        if tag.name == "malignant":
            malign_tag_id = tag.id
        else:
            benign_tag_id = tag.id
            
    threshold = 0.6
    
    malign_token_tag = PredictionQueryTag(id = malign_tag_id, min_threshold=threshold, max_threshold=1)
    malign_token = PredictionQueryToken(tags=[malign_token_tag], iteration_id = iteration_id, max_count = 100)
    new_images_malign = trainer.query_predictions(project_id, malign_token)

    benign_token_tag = PredictionQueryTag(id = benign_tag_id, min_threshold=threshold, max_threshold=1)
    benign_token = PredictionQueryToken(tags=[ benign_token_tag], iteration_id = iteration_id, max_count = 2 * len(new_images_malign.results))
    new_images_benign = trainer.query_predictions(project_id, benign_token)
    
    if (len(new_images_malign.results) > len(new_images_malign.results)):
        return False
    
    upload_batch_pred(new_images_malign, malign_tag_id)
    upload_batch_pred(new_images_benign, benign_tag_id)
    
    CustomBaseModelInfo(project_id=project_id, iteration_id= iteration_id)
    try:
        trainer.train_project(project_id, selected_tags=[benign_tag_id, malign_tag_id], notification_email_address="PMEDT@outlook.fr", force_train=True)
    except CustomVisionErrorException:
        return False
        
    return True

# upload to the training set, by batch of 64, a list (of a certain tag) of images.
def upload_batch_pred(query, tag):
    for i in range(0, len(query.results), 64):
        batch = []
        for image in query.results[i: i+ 64] :
            print(image.id)
            image_entry = ImageIdCreateEntry(id=image.id, tag_ids = [tag])
            batch.append(image_entry)
        batch = ImageIdCreateBatch(images=batch, tag_ids = [tag])
        trainer.create_images_from_predictions(project_id, batch)


In [5]:
retrain()

True

In [3]:
ressource_id = # set your own

In [95]:
 def publish_last_iteration():
    # Create the training client.
    trainer = CustomVisionTrainingClient(ENDPOINT, credentials)
    # Get the last iteration
    iterations = trainer.get_iterations(project_id)
    it_to_publish = iterations[0].id

    # Unpublish second last iteration
    if len(iterations) > 1:
        if iterations[1].status == "succeeded":
            trainer.unpublish_iteration(project_id, iterations[1].id)
    
    publish_name = "iteration 1"
    if len(iterations) > 0:
        second_last_name = iterations[1].name
        last_num = int(second_last_name.split(" ")[1])
        publish_name = "iteration " + str(last_num + 1)

    trainer.publish_iteration(project_id, it_to_publish, publish_name, ressource_id)


In [94]:
publish_last_iteration()